#### Check pictures in SPA och söka i SPA med folk kopplade till IFK Göteborg som saknar bild i WD P18
* [This notebook](https://github.com/salgo60/spa2Commons/blob/main/Notebook/SPA%20IFKDB.ipynb) *
* IFK Göteborg
  * [ifkdb.se](https://ifkdb.se)
  * Wikidata [property P11905](https://www.wikidata.org/wiki/Property:P11905)
      * GITHUB Issue [#12](https://github.com/salgo60/ifkdb/issues/12)
      * API SPA https://portrattarkiv.se/endpoints/search.php
      * [SPARQL söker fram gubbar i WD födda innan 1920](https://w.wiki/7Ayd)

In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2023-08-07 13:43:30.655973


In [2]:
import urllib3, json
import pandas as pd 
http = urllib3.PoolManager() 

url= "https://portrattarkiv.se/endpoints/search.php"  


In [3]:


import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

# IFKDB personer födda innan 1920 utan bild - SPA innehåller mest gamla personer
queryIFKDB = """SELECT (CONCAT (?itemLabel," ",str(year(?birthDate))) AS ?search) ?item ?itemLabel 
  (str(year(?birthDate)) AS ?year)
  WHERE {
  ?item wdt:P11905 ?ifkdb. 
  minus{?item wdt:P19 ?img} # saknar bild
  ?item wdt:P569 ?birthDate. 
  FILTER(xsd:integer(year(?birthDate)) < 1920)
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv", "en". }
} order by ?itemLabel """


 
def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

#SPAdf = get_sparql_dataframe(endpoint_url, query)
#SPAdf = get_sparql_dataframe(endpoint_url, queryNoPicNorra1890)
SPAdf = get_sparql_dataframe(endpoint_url, queryIFKDB)

SPAdf.info()   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   search     11 non-null     object
 1   item       11 non-null     object
 2   itemLabel  11 non-null     object
 3   year       11 non-null     object
dtypes: object(4)
memory usage: 480.0+ bytes


In [4]:
SPAdf.shape

(11, 4)

In [5]:
  SPAdf

,search,item,itemLabel,year
0,Douglas Krook 1897,http://www.wikidata.org/entity/Q1252363,Douglas Krook,1897
1,Elis Berndtsson 1902,http://www.wikidata.org/entity/Q121160724,Elis Berndtsson,1902
2,Erik Eiserman 1892,http://www.wikidata.org/entity/Q121160732,Erik Eiserman,1892
3,Gustaf Hermansson 1900,http://www.wikidata.org/entity/Q121160746,Gustaf Hermansson,1900
4,Gösta Bohman 1911,http://www.wikidata.org/entity/Q121160725,Gösta Bohman,1911
5,Göte Andersson 1919,http://www.wikidata.org/entity/Q121160713,Göte Andersson,1919
6,Helge Andersson 1899,http://www.wikidata.org/entity/Q121160714,Helge Andersson,1899
7,John Andersson 1914,http://www.wikidata.org/entity/Q121160715,John Andersson,1914
8,John Säwström 1886,http://www.wikidata.org/entity/Q121052611,John Säwström,1886
9,Olof Hjärn 1903,http://www.wikidata.org/entity/Q121160747,Olof Hjärn,1903


In [6]:
SPAdetail = "https://portrattarkiv.se/details/"
for index, row  in SPAdf.iterrows():
    print("\n",row["search"],row["item"])
    encoded_body = json.dumps({
        "limit": "5",
        "from": "0",
        "birthyear":row["year"],
        "all":row["search"]
    })
    
    r = http.request('POST', url,
                 headers={'Content-Type': 'application/json'},
                 body=encoded_body)

    if r.status != 200:
        print(r.status)
        continue
    
    data = json.loads(r.data.decode('utf-8'),) 
    urls = []


    for h in data["hits"]["hits"]:
        id = h["_id"]
        #print(h)
        source = h["_source"]
        try:
 #           urlPicture = urlbasePic + id
 #           urls.append(urlPicture)
            score = h["_score"]
            FirstName = source["FirstName"]
            LastName = source["LastName"]
            BirthYear = source["BirthYear"]
            print("\t\t",score,FirstName, " ", LastName, " - ", BirthYear,SPAdetail+id, "\t", )
        except:
            print("Error")



 Douglas Krook 1897 http://www.wikidata.org/entity/Q1252363
		 20.313782 Karl Arnold Douglas   Larsson  -  1897 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAACiN4A 	
		 19.261509 Albert (Abbe) Ragnar   Krook  -  1897 https://portrattarkiv.se/details/0piq-fX0h5AAAAAAAABkjw 	
		 19.255459 Douglas   Olsson  -  1897 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAACet2g 	
		 18.22902 Douglas   Bondeman  -  1897 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAA2JNg 	
		 18.155508 Douglas   Wallhäll  -  1897 https://portrattarkiv.se/details/ciLMGSqrYjAAAAAAAABicA 	

 Elis Berndtsson 1902 http://www.wikidata.org/entity/Q121160724
		 20.571924 Erland Aloys Ingvar   Berndtsson  -  1902 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAACiLnw 	
		 19.767786 Ellen   Nilsson-Berndtsson  -  1902 https://portrattarkiv.se/details/wbbNj8LLWMAAAAAAAAAoEA 	
		 18.01412 Carl   Berndtsson  -  1902 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABVsXw 	
		 17.49529 Elis   Nestor  -  1902 https://p